In [43]:
import seaborn as sns
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib as ml

In [44]:
from neurosynth.base.dataset import Dataset
dataset = Dataset.load("../data/0.6/datasets/db_v6_topics-60.pkl")

In [45]:
from copy import deepcopy
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import correlation

In [46]:
lfc_mask = nib.load('../masks/HO_ROIs/cortex_30.nii.gz')

In [47]:
masker = deepcopy(dataset.masker)

In [48]:
masker.add({'roi': lfc_mask})

In [49]:
roi_data = dataset.image_table.data[masker.get_mask(['voxels', 'roi']), :].toarray()

In [50]:
i, j, k = np.meshgrid(*map(np.arange, lfc_mask.get_data().shape), indexing='ij')
i -= i.max()/2

In [51]:
i = masker.mask(i)
j = masker.mask(j)
k = masker.mask(k)

In [52]:
contra_correlation = []
for row_ix, row in enumerate(roi_data):
    inter = set(np.where(i == (i[row_ix] * -1))[0]).intersection(
        set(np.where(j == j[row_ix])[0])).intersection(
        set(np.where(k == k[row_ix])[0]))
    if len(inter) == 1:
        ix = list(inter)[0]
        contra_data = roi_data[ix]
        contra_corr = correlation(contra_data, row)
    else:
        contra_corr = 0
    
    contra_correlation.append(contra_corr)

In [53]:
?correlation

In [54]:
(np.array(contra_correlation) ==  0).sum()

9419

In [68]:
contra_img = nib.Nifti1Image(masker.unmask(np.array(contra_correlation)), affine=lfc_mask.get_affine())

In [70]:
nib.save(contra_img, '../results/laterality/cortex_laterality.nii.gz')

### Display in PySurfer

In [71]:
%gui qt
from surfer import Brain, project_volume_data
from IPython.display import Image
import os
os.environ['SUBJECTS_DIR'] = '/Applications/freesurfer/subjects/'
os.environ['FREESURFER_HOME'] = '/Applications/freesurfer/'

In [72]:
from base import img_plot

In [81]:
brain = Brain('fsaverage', 'split', 'pial', cortex='bone', size=(780, 800), background='white',views=['lat', 'med'])
img_plot.display_bilateral(brain, contra_img, colormap=ml.pyplot.get_cmap('viridis').colors, max=1, thresh=0.7, min=0.7, discrete=False)
# out = '../reports/dissertation/figures/base_rate.png'
# brain.save_image(out)
# Image(out)

mri_vol2surf --mov /var/folders/k7/sjgnyj6j5z17gr14p3cnp0fc0000gn/T/tmpuu1ve1.nii.gz --hemi lh --surf white --regheader fsaverage --projfrac-max 0 1 0.1 --o /var/folders/k7/sjgnyj6j5z17gr14p3cnp0fc0000gn/T/pysurfer-v2s2M_O_C.mgz


INFO:surfer:mri_vol2surf --mov /var/folders/k7/sjgnyj6j5z17gr14p3cnp0fc0000gn/T/tmpuu1ve1.nii.gz --hemi lh --surf white --regheader fsaverage --projfrac-max 0 1 0.1 --o /var/folders/k7/sjgnyj6j5z17gr14p3cnp0fc0000gn/T/pysurfer-v2s2M_O_C.mgz


mri_vol2surf --mov /var/folders/k7/sjgnyj6j5z17gr14p3cnp0fc0000gn/T/tmpuu1ve1.nii.gz --hemi rh --surf white --regheader fsaverage --projfrac-max 0 1 0.1 --o /var/folders/k7/sjgnyj6j5z17gr14p3cnp0fc0000gn/T/pysurfer-v2sB9O2ET.mgz


INFO:surfer:mri_vol2surf --mov /var/folders/k7/sjgnyj6j5z17gr14p3cnp0fc0000gn/T/tmpuu1ve1.nii.gz --hemi rh --surf white --regheader fsaverage --projfrac-max 0 1 0.1 --o /var/folders/k7/sjgnyj6j5z17gr14p3cnp0fc0000gn/T/pysurfer-v2sB9O2ET.mgz
